In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.compose import ColumnTransformer
pd.set_option('display.max_columns', None)
# eye candy plots
plt.style.use('https://github.com/dhaitz/matplotlib-stylesheets/raw/master/pitayasmoothie-light.mplstyle')

In [2]:
df = pd.read_csv("../../data/processed/2_scored_pueblos.csv")
df.shape
df = df.dropna(subset=['koppen_climate'])

In [3]:
df

df.query("total_population < 6000", inplace=True)
df.shape

(6791, 48)

In [5]:
df.columns

Index(['Unnamed: 0', 'cmun', 'municipality', 'province',
       'autonomous_community', 'altitude', 'longitude', 'latitude',
       'towns_in_vicinity', '0-17', '18-24', '25-34', '35-54', '55+', 'male',
       'female', 'total_population', 'nearest_airport',
       'closest_distance_airport', 'nearest_train_station',
       'closest_distance_train', 'closest_distance_hospital',
       'nearest_hospital', 'closest_distance_school', 'nearest_school',
       'koppen_climate', 'description', 'number_of_houses',
       'count_of_stations', 'suburban_train', 'category_town_size',
       'score_connectivity', 'category_connectivity', 'score_economy',
       'score_area_economy', 'category_economy', 'score_hospital_distance',
       'score_area_hospital', 'score_school_distance', 'score_area_school',
       'score_transport', '0-17_pct', '18-24_pct', '25-34_pct', '35-54_pct',
       '55+_pct', 'score_age', 'cagetory_town_age'],
      dtype='object')

In [4]:
df = df.drop(['cmun', 'municipality', 'towns_in_vicinity','province', 'autonomous_community', 'altitude','connectivity_category', 'description', 'closest_distance_airport', 'nearest_airport','nearest_train_station', 'closest_distance_train', 'closest_distance_hospital', 'nearest_hospital', 'closest_distance_school', 'nearest_school', 'longitude', 'latitude', 'male'], axis=1)

KeyError: "['connectivity_category'] not found in axis"

In [ ]:
ohe = OneHotEncoder(drop='first', sparse_output=False)
scaler = StandardScaler()

pipeline = Pipeline(steps=[
    ('ohe', OneHotEncoder(drop='first', sparse_output=False))
])

pipeline.fit(df[['koppen_climate', 'town_size', 'final_age_category']])

Pipeline(steps=[('ohe', OneHotEncoder(drop='first', sparse_output=False))])

In [ ]:
fe_transformer = ColumnTransformer(transformers=[
    ('transf_cat', pipeline, ['koppen_climate']),
    ('scaled', scaler, [
         'connectivity_score',
        'economy_score', 'economy_score_area',
        'hospital_distance_score', 'school_distance_score', 'hospital_score_area',
        'school_score_area', 'train_distance_score', 'airport_distance_score', 'transport_score'
    ])
], remainder='drop')

fe_transformer.fit(df)

ColumnTransformer(transformers=[('transf_cat',
                                 Pipeline(steps=[('ohe',
                                                  OneHotEncoder(drop='first',
                                                                sparse_output=False))]),
                                 ['koppen_climate']),
                                ('scaled', StandardScaler(),
                                 ['connectivity_score', 'economy_score',
                                  'economy_score_area',
                                  'hospital_distance_score',
                                  'school_distance_score',
                                  'hospital_score_area', 'school_score_area',
                                  'train_distance_score',
                                  'airport_distance_score',
                                  'transport_score'])])

In [ ]:
df_train = fe_transformer.transform(df)


(6670, 15)

In [ ]:
df_train.to_csv("../../data/processed/cluster_model_train.csv", index=False)